In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
file_path = "../../queenCsvOut.csv"
df = pd.read_csv(file_path, skiprows=[1])

In [3]:
df

,Dati,Time,CARGO,CARGO_PAX,DEPTH,ENGINE_1_FLOWRATE,ENGINE_1_FLOWRATEA,ENGINE_1_FLOWRATEB,ENGINE_1_FLOWTEMPA,ENGINE_1_FLOWTEMPB,...,STW,THRUST_1,THRUST_2,TORQUE_1,TORQUE_2,TRACK_MADE_GOOD,WIND_ANGLE,WIND_SPEED,WIND_ANGLE_TRUE,WIND_SPEED_TRUE
0,190830_000000,0.0,1.000000e+09,1.0,0.0049,-195.7947,2874.4858,2678.6912,30.7517,35.60,...,0.0100,173.1906,0.0,61.7668,0.0,192.5550,193.0,5.8283,16.3723,5.8379
1,190830_000100,1.0,1.000000e+09,1.0,0.0050,-196.1073,2875.3389,2679.2317,30.6250,35.60,...,0.0283,173.4646,0.0,61.8693,0.0,181.2967,221.0,8.2265,44.3432,8.2348
2,190830_000200,2.0,1.000000e+09,1.0,0.0050,-188.3371,2911.7334,2723.3962,30.4967,35.60,...,0.0467,118.5395,0.0,44.2674,0.0,166.5850,221.0,4.9133,44.1781,4.9241
3,190830_000300,3.0,1.000000e+09,1.0,0.0050,-196.6828,2932.1985,2735.5156,30.4000,35.60,...,0.0467,94.1722,0.0,36.4697,0.0,187.0533,203.0,6.1450,26.2657,6.1525
4,190830_000400,4.0,1.000000e+09,1.0,0.0051,-230.8546,2908.0278,2677.1733,30.3567,35.52,...,0.1200,78.4188,0.0,35.9350,0.0,119.7333,209.0,4.4600,30.9895,4.6435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098849,210930_225300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098850,210930_225400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098851,210930_225500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098852,210930_225600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns, len(df.columns)

(Index(['Dati', 'Time', 'CARGO', 'CARGO_PAX', 'DEPTH', 'ENGINE_1_FLOWRATE',
        'ENGINE_1_FLOWRATEA', 'ENGINE_1_FLOWRATEB', 'ENGINE_1_FLOWTEMPA',
        'ENGINE_1_FLOWTEMPB', 'ENGINE_1_FUEL_CONSUMPTION', 'ENGINE_2_FLOWRATE',
        'ENGINE_2_FLOWRATEA', 'ENGINE_2_FLOWRATEB', 'ENGINE_2_FLOWTEMPA',
        'ENGINE_2_FLOWTEMPB', 'ENGINE_2_FUEL_CONSUMPTION', 'HEADING',
        'LATITUDE', 'LONGITUDE', 'PAX', 'PITCH_1', 'PITCH_2', 'POWER_1',
        'POWER_2', 'RATE_OF_TURN', 'SOG', 'SOG_SPEEDLOG_LONG',
        'SOG_SPEEDLOG_TRANS', 'SPEED_1', 'SPEED_2', 'STW', 'THRUST_1',
        'THRUST_2', 'TORQUE_1', 'TORQUE_2', 'TRACK_MADE_GOOD', 'WIND_ANGLE',
        'WIND_SPEED', 'WIND_ANGLE_TRUE', 'WIND_SPEED_TRUE'],
       dtype='object'),
 41)

In [5]:
# trip id to identify which trip the records belong to
df["trip_id"] = df["Dati"].apply(lambda x: x.split("_")[0]).astype(str)
df = df[[df.columns[-1]] + list(df.columns[:-1])]
df.trip_id.nunique()

763

In [6]:
df.trip_id

0          190830
1          190830
2          190830
3          190830
4          190830
            ...  
1098849    210930
1098850    210930
1098851    210930
1098852    210930
1098853    210930
Name: trip_id, Length: 1098854, dtype: object

In [7]:
# naive mode definition
# TODO: add more constraints to get a more accurate mode
df["SPEED_1"] = df["SPEED_1"].astype(float)
df["SPEED_2"] = df["SPEED_2"].astype(float)

# MODE for operation mode: 1 for mode1, 2 for mode2, 0 for unknown(speed with nan)
def naive_operation_mode(row):
    if pd.isna(row['SPEED_1']) or (pd.isna(row["SPEED_2"])):
        return 0
    elif ((row["SPEED_1"]<1) ^ (row["SPEED_2"]<1)):
        return 1
    else:
        return 2

df["MODE"] = df.apply(naive_operation_mode, axis=1)

In [8]:
df.groupby("MODE").count()

,trip_id,Dati,Time,CARGO,CARGO_PAX,DEPTH,ENGINE_1_FLOWRATE,ENGINE_1_FLOWRATEA,ENGINE_1_FLOWRATEB,ENGINE_1_FLOWTEMPA,...,STW,THRUST_1,THRUST_2,TORQUE_1,TORQUE_2,TRACK_MADE_GOOD,WIND_ANGLE,WIND_SPEED,WIND_ANGLE_TRUE,WIND_SPEED_TRUE
MODE,,,,,,,,,,,,,,,,,,,,,
0,262909,262909,262713,5,5,0,0,0,0,0,...,0,0,0,0,0,5,0,0,0,0
1,381318,381318,381318,373160,373160,381318,381318,381318,381318,381318,...,381318,381318,381318,381318,381318,373160,381318,381318,380702,380702
2,454627,454627,454627,450222,450222,454627,454627,454627,454627,454627,...,454627,454627,454627,454627,454627,450222,454627,454627,406502,406527


In [9]:
df[pd.isna(df["Time"])].trip_id.unique(), len(df[pd.isna(df["Time"])])

(array(['210930'], dtype=object), 196)

In [10]:
df[pd.isna(df["Time"])].Dati.min(), df[pd.isna(df["Time"])].Dati.max()

('210930_194200', '210930_225700')

In [11]:
# drop the 196 cols from trip id 210930 where Time is empty
df.dropna(subset=["Time"], inplace=True)

In [12]:
df[pd.isna(df["Time"])]

,trip_id,Dati,Time,CARGO,CARGO_PAX,DEPTH,ENGINE_1_FLOWRATE,ENGINE_1_FLOWRATEA,ENGINE_1_FLOWRATEB,ENGINE_1_FLOWTEMPA,...,THRUST_1,THRUST_2,TORQUE_1,TORQUE_2,TRACK_MADE_GOOD,WIND_ANGLE,WIND_SPEED,WIND_ANGLE_TRUE,WIND_SPEED_TRUE,MODE


In [13]:
# from the Time variable, get the corresponding datetime, day of the year, hour, and day of the week
STARTING_DT = datetime.datetime.now()
def get_dt_from_min(minute, dt):
    dt = dt + datetime.timedelta(minutes=minute) # days, seconds, then other fields.
    day = dt.day
    hour = dt.hour
    weekday = dt.weekday()
    return dt, day, hour, weekday
df["datetime"], df["day"], df["hour"], df["weekday"] = zip(*df["Time"].apply(get_dt_from_min, dt=STARTING_DT))

In [14]:
df[["datetime", "day", "hour", "weekday"]]

,datetime,day,hour,weekday
0,2023-05-17 22:21:12.489996,17,22,2
1,2023-05-17 22:22:12.489996,17,22,2
2,2023-05-17 22:23:12.489996,17,22,2
3,2023-05-17 22:24:12.489996,17,22,2
4,2023-05-17 22:25:12.489996,17,22,2
...,...,...,...,...
1098653,2025-06-18 21:14:12.489996,18,21,2
1098654,2025-06-18 21:15:12.489996,18,21,2
1098655,2025-06-18 21:16:12.489996,18,21,2
1098656,2025-06-18 21:17:12.489996,18,21,2
